In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import collections
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from imblearn.over_sampling import SMOTE 
from sklearn import svm

In [0]:
train_data = pd.read_csv('./drive/My Drive/BT4240 data/result.csv')\
              .drop(columns=['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
train_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.00,Supplies,My students need matching shirts to wear for d...
1,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,1,{Amazon - Fire Kids Edition - 7'- Tablet - 16G...,727.36,Technology,My students need tablets in order to communic...
2,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,Math & Science,Mathematics,Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,1,"{Kore Patented WOBBLE Chair, Made in the USA, ...",319.80,Supplies,My students need seating that allows the most ...
3,p030093,b9e731e16ad8669f37a43e5316518106,Teacher,VA,2017-03-09 17:16:13,Grades PreK-2,"Literacy & Language, Math & Science","Literature & Writing, Mathematics","Writing, Writing, Writing!",My classroom has 24 students in it. We have an...,"In our classroom, we use dry erase boards thro...",NaN,NaN,My students need individual dry erase boards t...,1,1,{BOARD LEARNING THE CLASSICS W/MARKER},102.00,Supplies,My students need individual dry erase boards t...
4,p225747,4681d60136ea859084ab488f2254b219,Ms.,PA,2016-12-08 13:25:03,Grades 3-5,"Applied Learning, Literacy & Language","Character Education, Literacy",Life Lessons through Literacy,"My school is located in a high poverty area, j...",As a Reading Specialist who serves Grades 1-8 ...,NaN,NaN,My students need books that teach life lessons...,3,1,"{A Bad Case of Stripes (Scholastic Bookshelf),...",176.59,Books,My students need books that teach life lessons...


In [0]:
train_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'description', 'total_cost', 'Project Resource Category',
       'Project Need Statement'],
      dtype='object')

In [0]:
train_data['project_subject_categories'] = train_data['project_subject_categories'].str.split(", ").to_frame()
train_data['project_subject_subcategories'] = train_data['project_subject_subcategories'].str.split(", ").to_frame()
train_data['Project Resource Category'] = train_data['Project Resource Category'].str.split(", ").to_frame()

train_data.head(1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"[Music & The Arts, Health & Sports]","[Performing Arts, Team Sports]",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.0,[Supplies],My students need matching shirts to wear for d...


In [0]:
train_data['project_subject_categories'] = train_data['project_subject_categories'].apply(collections.Counter)
train_data['project_subject_subcategories'] = train_data['project_subject_subcategories'].apply(collections.Counter)
train_data['Project Resource Category'] = train_data['Project Resource Category'].apply(collections.Counter)

project_subject_categories = pd.DataFrame.from_records(train_data['project_subject_categories']).fillna(value=0).add_prefix("proj_cat_")
project_subject_subcategories = pd.DataFrame.from_records(train_data['project_subject_subcategories']).fillna(value=0).add_prefix("proj_sub_cat_")
Project_Resource_Category = pd.DataFrame.from_records(train_data['Project Resource Category']).fillna(value=0).add_prefix("res_cat_")


train_data = pd.concat([train_data, project_subject_categories], axis=1)
train_data = pd.concat([train_data, project_subject_subcategories], axis=1)
train_data = pd.concat([train_data, Project_Resource_Category], axis=1)

train_data.head(1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Care & Hunger,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Special Needs,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,proj_sub_cat_Warmth,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"{'Music & The Arts': 1, 'Health & Sports': 1}","{'Performing Arts': 1, 'Team Sports': 1}",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.0,{'Supplies': 1},My students need matching shirts to wear for d...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [0]:
train_data.plot

In [0]:
print(len(train_data.columns))
print(len(set(train_data.columns)))
!pip install iteration_utilities
from iteration_utilities import duplicates
list(duplicates(train_data.columns))

77
77
     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for iteration-utilities: filename=iteration_utilities-0.7.0-cp36-cp36m-linux_x86_64.whl size=189310 sha256=06b6c7728181d5b1b5a052b02f031aca9ed8a9f3f0c4045de4b8b7e063a00655
  Stored in directory: /root/.cache/pip/wheels/41/ea/fd/de8371ff85ff0aafd560d50a77ca22d107117094d660131e2a
Successfully built iteration-utilities


[]

In [0]:
train_data = train_data.loc[:,~train_data.columns.duplicated()]
train_data = train_data.drop(columns = ['proj_sub_cat_Care & Hunger', 'proj_sub_cat_Special Needs', 'proj_sub_cat_Warmth'])
len(train_data.columns)

74

In [0]:
train_df = train_data.select_dtypes(exclude=['object','category'])
print(len(train_df.columns))
print(train_df.dtypes.unique())
cat_columns = [col for col in train_df.columns if 'cat' in col]
train_df.head(1)

57
[dtype('int64') dtype('float64')]


,teacher_number_of_previously_posted_projects,project_is_approved,total_cost,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,1,0,400.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [0]:
train_df[cat_columns] = train_df[cat_columns].astype('int8')
train_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,teacher_number_of_previously_posted_projects,project_is_approved,total_cost,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,1,0,400.00,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,727.36,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,13,1,319.80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,1,102.00,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,3,1,176.59,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X = train_df.loc[:, train_df.columns != 'project_is_approved']
y = train_df.loc[:, train_df.columns == 'project_is_approved']
print(X.shape)
print(y.shape)

(121308, 56)
(121308, 1)


In [0]:
print("Number of features: {}".format(len(X.columns)))

Number of features: 56


In [0]:
# standardize numeric features: teacher_number_of_previously_posted_projects, total_cost
scaler = StandardScaler()
stand_cols = ['teacher_number_of_previously_posted_projects', 'total_cost']
for col_name in stand_cols:
  X[col_name] = scaler.fit_transform(X[col_name].values.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
# split X, Y to train, valid and test by 6:2:2
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.25, random_state=1)
print("Train: {}, Validation: {}, Test: {}".format(X_train.shape, X_val.shape, X_test.shape))

Train: (77636, 56), Validation: (19410, 56), Test: (24262, 56)


In [0]:
y_val['project_is_approved'].value_counts()

1    17661
0     1749
Name: project_is_approved, dtype: int64

In [0]:
X_train.describe(include='all')

,teacher_number_of_previously_posted_projects,total_cost,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
count,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000,77636.000000
mean,0.000788,-0.001093,0.102414,0.013125,0.099760,0.048547,0.530823,0.431617,0.085025,0.110258,0.013125,0.089855,0.015212,0.007162,0.026727,0.002975,0.040175,0.035924,0.001778,0.052051,0.006685,0.005294,0.006852,0.031802,0.039196,0.071835,0.027758,0.335940,0.234582,0.314455,0.026817,0.008682,0.017981,0.005178,0.015701,0.014336,0.015663,0.051265,0.000064,0.124053,0.000180,0.000425,0.024125,0.000515,0.000129,0.000721,0.000425,0.000747,0.000155,0.000090,0.006530,0.000515,0.000077,0.445399,0.396762,0.000026
std,1.000075,1.001156,0.303194,0.113812,0.299682,0.214921,0.499052,0.495305,0.278921,0.313213,0.113812,0.285976,0.122396,0.084323,0.161286,0.054467,0.196370,0.186102,0.042123,0.222131,0.081489,0.072567,0.082496,0.175474,0.194062,0.258217,0.164279,0.472321,0.423740,0.464301,0.161551,0.092770,0.132884,0.071772,0.124319,0.118873,0.124168,0.220539,0.008025,0.329644,0.013428,0.020613,0.153439,0.022693,0.011349,0.026848,0.020613,0.027323,0.012432,0.009495,0.080548,0.022693,0.008791,0.497013,0.489229,0.005076
min,-0.408574,-0.834264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.408574,-0.554612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [0]:
# use SMOTE with only the train set
os = SMOTE(random_state=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['project_is_approved'])

# we can check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of not approved in oversampled data",len(os_data_y[os_data_y['project_is_approved']==0]))
print("Number of approved in oversampled data",len(os_data_y[os_data_y['project_is_approved']==1]))
print("Proportion of not approved data in oversampled data is ",len(os_data_y[os_data_y['project_is_approved']==0])/len(os_data_X))
print("Proportion of approved data in oversampled data is ",len(os_data_y[os_data_y['project_is_approved']==1])/len(os_data_X))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


length of oversampled data is  141196
Number of not approved in oversampled data 70598
Number of approved in oversampled data 70598
Proportion of not approved data in oversampled data is  0.5
Proportion of approved data in oversampled data is  0.5


In [0]:
svm = svm.SVC(kernel = 'linear')
svm.fit(os_data_X, os_data_y)
print('Accuracy of SVM classifier on validation set: {:.2f}'.format(svm.score(X_val, y_val)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
y_val_pred = svm.predict(X_val)



Recursive Feature Elimination Later 

In [0]:
# Recursive Feature Elimination
# To select features by recursively considering smaller and smaller sets of features.

data_final_vars = train_df.columns.values.tolist()
y=['project_is_approved']
X=[i for i in data_final_vars if i not in y]

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [0]:
from itertools import compress
cols = list(compress(X,rfe.support_))

In [0]:
X=os_data_X[cols]
y=os_data_y['project_is_approved']